### leetcode 550

이 문제는 첫 번째 로그인 날짜 다음 날에 다시 로그인한 플레이어의 비율을 리턴하는 문제.

연속적인 날짜를 계산하는 알고리즘이 필요함.


### 아이디어

`서브 쿼리로 min_date(첫 번째 로그인 한 날)를 구하고 접속한 날을 뺏을 때 1 이라면 연속적으로 접속했구나(at least 2 day)`

### 쿼리

```python
select
round(count(distinct player_id) / (select count(distinct player_id) from activity), 2) as fraction

from (select 
      player_id, event_date,
      min(event_date) over(partition by player_id order by event_date) as consecutive_logs
      from activity) as sub_table
      
where datediff(event_date, consecutive_logs) = 1
```

### 살펴보기

먼저 partition by 와 group by 를 비교해보면. 

- group by 는 여러 개의 컬럼이 있을 때 상세한 값들이 한 개로 줄여버림 (min, max, sum 값을 구할 때 용이) 

- partition by 는 줄이는 것이 아니고 그룹 별로 묶고 각 컬럼의 특징을 살림.

표로 담기 어려워서 차이점을 구글링 해보면 좋을 것 같음 (보면 이해가 감)

중요하다 생각하는 부분 : `min(event_date) over(partition by player_id order by event_date) as consecutive_logs`

`player_id 별로 파티션을 나누고 event_date 를 오름차순으로 정렬한 상태에서 최초 로그인 날짜를 뽑아냄.`

그래서 event_date 와 뺏을 때 그 차이가 1 일 때 필터링.

필터링된 데이터에서 player_id 는 distinct 가 아니고 레코드가 여러 줄인 상태이기 때문에 

남아있는 distinct player_id 를 카운팅하고 전체 table 의 distinct player_id 의 수로 나누면 

문제에서 원하는 비율을 얻을 수 있다.